# Progetto PAA, a.a. 2015/2016

Autori: Tommaso Papini (5537529), Gabriele Bani (5719258)

## Quicksort

## Mergesort
Il **Mergesort** è un algoritmo di ordinamento ricorsivo, basato su confronti, che utilizza la tecnica del **divide et impera** per ordinare, ad esempio, un vettore di n elementi.

Supponiamo di voler ordinare un vettore $a[1..n]$. Secondo l'implementazione del Mergesort vista a lezione, occorre:

<ul>
<li> suddividere, ricorsivamente, il vettore a in due parti fino ad arrivare all'insieme contenente un solo elemento</li>
<li> ordinare e fondere i sottovettori partendo dai sottoinsiemi che hanno il minor numero di elementi
</ul>

Il numero di confronti del **Mergesort** è dato dalla seguente relazione di ricorrenza:

$$
C_n = C_{\lfloor\frac{n}{2}\rfloor} + C_{\lceil\frac{n}{2}\rceil} + n\\
C_1 = 0
$$

Verifichiamo adesso che, se $n = 2^m$, allora $$C_n = n \lfloor log_2 n\rfloor - 2^{\lfloor log_2 n\rfloor +1} + 2n$$